# Prebuilt ReAct Agent - Structured Output

## Define tools explicitly

In [ ]:
import requests
import yfinance as yf
from pprint import pformat
from langchain_core.tools import tool

@tool("lookup_stock")
def lookup_stock_symbol(company_name: str) -> str:
    """
    Converts a company name to its stock symbol using a financial API.

    Parameters:
        company_name (str): The full company name (e.g., 'Tesla').

    Returns:
        str: The stock symbol (e.g., 'TSLA') or an error message.
    """
    api_url = "https://www.alphavantage.co/query"
    params = {
        "function": "SYMBOL_SEARCH",
        "keywords": company_name,
        "apikey": "your_alphavantage_api_key"
    }
    
    response = requests.get(api_url, params=params)
    data = response.json()
    
    if "bestMatches" in data and data["bestMatches"]:
        return data["bestMatches"][0]["1. symbol"]
    else:
        return f"Symbol not found for {company_name}."


@tool("fetch_stock_data")
def fetch_stock_data_raw(stock_symbol: str) -> dict:
    """
    Fetches comprehensive stock data for a given symbol and returns it as a combined dictionary.

    Parameters:
        stock_symbol (str): The stock ticker symbol (e.g., 'TSLA').
        period (str): The period to analyze (e.g., '1mo', '3mo', '1y').

    Returns:
        dict: A dictionary combining general stock info and historical market data.
    """
    period = "1mo"
    try:
        stock = yf.Ticker(stock_symbol)

        # Retrieve general stock info and historical market data
        stock_info = stock.info  # Basic company and stock data
        stock_history = stock.history(period=period).to_dict()  # Historical OHLCV data

        # Combine both into a single dictionary
        combined_data = {
            "stock_symbol": stock_symbol,
            "info": stock_info,
            "history": stock_history
        }

        return pformat(combined_data)

    except Exception as e:
        return {"error": f"Error fetching stock data for {stock_symbol}: {str(e)}"}

## Define the basic agent

In [ ]:
from langgraph.prebuilt import create_react_agent
from langchain_openai import ChatOpenAI
from IPython.display import Image, display

agent = create_react_agent(
    model=ChatOpenAI(model="gpt-4o-mini"),
    tools=[lookup_stock_symbol, fetch_stock_data_raw],
    prompt="You are a financial advisor. Give clear analysis of risks and opportunities."
)

display(Image(agent.get_graph().draw_mermaid_png()))

In [ ]:
response = agent.invoke({"messages": [HumanMessage(content="Analyze stock symbol TSLA and provide a financial summary.")]})
for message in response['messages']:
    message.pretty_print()

## Structured Output

In [ ]:
from pydantic import BaseModel, Field
from typing import List, Optional

class FinancialInfo(BaseModel):
    company_name: str = Field(
        description="The full name of the company being analyzed."
    )
    stock_symbol: str = Field(
        description="The stock ticker symbol for the company (e.g., 'AAPL' for Apple)."
    )
    current_price: float = Field(
        description="The most recent stock price in USD."
    )
    market_cap: Optional[float] = Field(
        description="The market capitalization of the company in billions USD.",
        default=None
    )
    summary: str = Field(
        description="A brief 2-3 sentence summary of the company's current financial status and recent performance."
    )
    risk_assessment: str = Field(
        description="Assessment of investment risk as 'Low', 'Moderate', or 'High' with brief explanation."
    )

agent = create_react_agent(
    model=ChatOpenAI(model="gpt-4o-mini"),
    tools=[lookup_stock_symbol, fetch_stock_data_raw],
    prompt="You are a financial advisor. Analyze the requested company and provide a structured assessment.",
    response_format=FinancialInfo,
    version="v2"
)

display(Image(agent.get_graph().draw_mermaid_png()))

In [ ]:
from langchain_core.messages import HumanMessage
import json

response = agent.invoke({"messages": [HumanMessage(content="Analyze stock symbol TSLA and provide a financial summary.")]})
structured_output = response['structured_response']
print(json.dumps(structured_output.model_dump(), indent=2))

The graph will make a separate call to the LLM to generate the structured response after the agent loop is finished. Check traces in LangSmith.

### Problems with Using a Separate "generate_structured_response" Node

1. **Extra API Cost**: An additional LLM call that could be avoided.

2. **Less Agent Control**: The agent can't decide when or if structured output should be generated - it's automatic.

3. **Workflow Rigidity**: Output generation happens regardless of whether sufficient data was collected.


Possible workaround - use the FinancialInfo as a tool as its final step, so then it creates the structured output while context is still there.

In [ ]:
system_prompt = """You are a financial advisor. Analyze the requested company and provide a structured assessment.

IMPORTANT: When you have gathered all necessary data, call the FinancialInfo tool with your findings.
After receiving the structured data from the FinancialInfo tool, DO NOT add any additional commentary.
Simply return the structured data as your final response."""

agent = create_react_agent(
    model=ChatOpenAI(model="gpt-4o-mini"),
    tools=[lookup_stock_symbol, fetch_stock_data_raw, FinancialInfo],
    prompt=system_prompt
)

display(Image(agent.get_graph().draw_mermaid_png()))

In [ ]:
from langchain_core.messages import HumanMessage

response = agent.invoke({"messages": [HumanMessage(content="Analyze stock symbol TSLA and provide a financial summary.")]})

for message in response['messages']:
    message.pretty_print()

In [ ]:
from typing import Optional
from langgraph.graph import MessagesState

def post_model_hook(state):
    """Extract structured data from FinancialInfo tool calls"""
    latest_message = state["messages"][-1]
    
    if hasattr(latest_message, "tool_calls"):
        for tool_call in latest_message.tool_calls:
            if tool_call["name"] == "FinancialInfo":
                structured_data = tool_call["args"]
                return {"structured_response": FinancialInfo(**structured_data)}
    
    return {}


# we need our own state so we can add a new attribute "structured_response" that will be recognized by "post_model_hook"
class AgentState(MessagesState):
    remaining_steps: int    
    structured_response: Optional[FinancialInfo] = None


system_message = """You are a financial advisor. Analyze the requested company and provide a structured response using FinancialInfo tool."""

agent = create_react_agent(
    model=ChatOpenAI(model="gpt-4o-mini"),
    tools=[lookup_stock_symbol, fetch_stock_data_raw, FinancialInfo],
    prompt=system_message,

    post_model_hook=post_model_hook,
    version="v2",
    state_schema=AgentState
)

display(Image(agent.get_graph().draw_mermaid_png()))

In [ ]:
from langchain_core.messages import HumanMessage
import json

response = agent.invoke({"messages": [HumanMessage(content="Analyze stock symbol TSLA and provide a financial summary.")]})
structured_output = response.get('structured_response')
print(json.dumps(structured_output.model_dump(), indent=2))

## Reference Links

**1. LangGraph create_react_agent API Reference**

https://langchain-ai.github.io/langgraph/reference/agents/#langgraph.prebuilt.chat_agent_executor.create_react_agent

→ Technical reference for the create_react_agent function, including parameters, return types, and implementation details.


**2. LangGraph Structured Output Configuration**

https://langchain-ai.github.io/langgraph/agents/agents/#6-configure-structured-output

→ Documentation on configuring structured output in LangGraph agents, including response_format and output parsing options.


**3. LangGraph ReAct Agent with Structured Output Tutorial**

https://langchain-ai.github.io/langgraph/how-tos/react-agent-structured-output/

→ Step-by-step guide on implementing structured output with ReAct agents, including examples and best practices.